In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [9]:
!pip install chromedriver-autoinstaller

In [11]:
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()

'C:\\Users\\khb16\\anaconda3\\lib\\site-packages\\chromedriver_autoinstaller\\99\\chromedriver.exe'

In [2]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [3]:
html = 'https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=Detail&PublishMonth=0&SortOrder=2&page=1&Stockstatus=0&PublishDay=84&CustReviewRankStart=0&CustReviewCountStart=0&PriceFilterMax=-1&CID=1&SearchOption='
driver.get(html)
time.sleep(2)

In [7]:
#results = []

for page in range(1,501):
    url = "https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=Detail\
    &PublishMonth=0&SortOrder=2&page={}&Stockstatus=0&PublishDay=84&CustReviewRankStart=0&CustReviewCountStart=0&PriceFilterMax=-1\
    &CID=1&SearchOption=".format(page)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('ul > li > a.bo3')
    links = []
    for book_link in book_links:
        a= book_link['href']
        links.append(a)
    for link in links:
        driver.get(link)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            ISBN = soup.select('div.conts_info_list1 > ul > li')
            ISBN = ISBN[-1].text[-13:]
        except:
            pass

        #data = [ISBN]
        results.append(ISBN)

KeyboardInterrupt: 

In [8]:
len(results)

1949

In [14]:
ISBN123 = pd.DataFrame(results)

In [16]:
import pickle
import pandas as pd
ISBN123.to_pickle('ISBN123.plk')

In [17]:
ISBN123.head()

,0
0,9791192186313
1,9791189327156
2,9788954685672
3,9788954683371
4,9791191891072


In [18]:
import requests
import pandas as pd
import json

In [19]:
# API 호출키
KEY = 'ttbsueji951535001'

In [25]:
results[1]

'9791189327156'

In [23]:
url_look_list = []

for isbn in results:
    url_look = 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={}&itemIdType=isbn13&ItemId={}&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList'.format(KEY, isbn)
    url_look_list.append(url_look)
print(url_look_list)

['https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9791192186313&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9791189327156&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9788954685672&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9788954683371&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&It

In [26]:
items = []
lists = []
for url_look in url_look_list:
    try:
        response = requests.get(url_look)
        text = json.loads(response.text)
        #time.sleep(1)
        title = text['item'][0]['title']
        author = text['item'][0]['author']
        pubDate = text['item'][0]['pubDate']
        description = text['item'][0]['description']
        isbn13 = text['item'][0]['isbn13']
        categoryID = text['item'][0]['categoryId']
        categoryName = text['item'][0]['categoryName']
        salesPoint = text['item'][0]['salesPoint']
        adult = text['item'][0]['adult']
        rank = text['item'][0]['customerReviewRank']
        fullDes = text['item'][0]['fullDescription']
        fullDes2 = text['item'][0]['fullDescription2']
        story = text['item'][0]['subInfo']['story']
        try:
            reviewrank = text['item'][0]['reviewList'][0]['reviewRank']
            reviewtitle = text['item'][0]['reviewList'][0]['title']
            review_w = text['item'][0]['reviewList'][0]['writer']
        except:
            reviewrank = 0
            reviewtitle = 0
            review_w = 0
        lists.append([title, author, pubDate, description,isbn13,categoryID,categoryName,salesPoint,adult,rank,fullDes,fullDes2,story,reviewrank,reviewtitle,review_w])
    except:
        pass
cols = ['title', 'author', 'pubDate','description','isbn13','categoryID','categoryName','salesPoint','adult','rank','fullDes','fullDes2','story','reviewrank','reviewtitle','review_w']
df = pd.DataFrame(lists, columns = cols)

In [27]:
df

,title,author,pubDate,description,isbn13,categoryID,categoryName,salesPoint,adult,rank,fullDes,fullDes2,story,reviewrank,reviewtitle,review_w
0,시맨틱 에러 포토에세이,"왓챠, 래몽래인 (지은이)",2022-03-20,왓챠 오리지널 드라마 ‘시맨틱 에러’ 포토에세이. 베스트셀러이자 스테디셀러인 원작의...,9791192186313,51376,국내도서>에세이>사진/그림 에세이,242520,False,10,왓챠 오리지널 드라마 ‘시맨틱 에러’ 포토에세이. 베스트셀러이자 스테디셀러인 원작의...,<b>론칭 즉시 왓챠 1위! <BR>\n전체 ott 콘텐츠 트렌드 탑2 등극<BR>...,,10,[100자평]안봤지만 만점갈겨 안봐도 극락...♡ 초판본아니면죽음을,뱅슈리a
1,"물고기는 존재하지 않는다 - 상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기","룰루 밀러 (지은이), 정지인 (옮긴이)",2021-12-17,집착에 가까울 만큼 자연계에 질서를 부여하려 했던 19세기 어느 과학자의 삶을 흥미...,9791189327156,51002,국내도서>과학>기초과학/교양과학,325120,False,9,‘방송계의 퓰리처상’으로 불리는 피버디상(Peabody Awards)을 수상한 과학...,"<b>《워싱턴포스트》, 《북라이엇》, 《내서널퍼블릭라디오NPR》, 《시카고 트리뷴》...",,4,[마이리뷰]잘 쓴 책이지만 좋은 과학책은 ..,mogulkor
2,오래되고 멋진 클래식 레코드,"무라카미 하루키 (지은이), 홍은주 (옮긴이)",2022-03-25,"본업인 소설가 외에도 사시사철 음악과 함께하는 애호가, 눈에 들어온 것은 저도 모르...",9788954685672,51373,국내도서>에세이>외국에세이,45050,False,9,"본업인 소설가 외에도 사시사철 음악과 함께하는 애호가, 눈에 들어온 것은 저도 모르...","<b>● “자랑할 정도는 아니지만, 이런 걸 모았답니다.”<BR>\n 60년째 ...",,10,[100자평]기다리는 시간도 즐겁다. 빨리 오기를,마젠타
3,H마트에서 울다,"미셸 자우너 (지은이), 정혜윤 (옮긴이)",2022-02-28,그래미 어워드 후보에 두 번 오른 인디 팝 밴드 재패니즈 브렉퍼스트의 가수이자 기타...,9788954683371,51373,국내도서>에세이>외국에세이,34810,False,10,그래미 어워드 후보에 두 번 오른 인디 팝 밴드 재패니즈 브렉퍼스트의 가수이자 기타...,<b>“엄마가 이제 내 곁에 없는데 내가 한국인일 수 있을까?”<BR>\n<BR>\...,,10,[100자평]지난해 제가 경험했던 무수한 감정들과 다시 만나야 하는...,riri
4,나에게 고맙다 - 30만 부 기념 전면개정판,전승환 (지은이),2022-02-18,2016년 출간 이후 30만 부 넘게 판매된 힐링 에세이 &lt;나에게 고맙다&gt...,9791191891072,51371,국내도서>에세이>한국에세이,26825,False,10,2016년 출간 이후 30만 부 넘게 판매된 힐링 에세이 &lt;나에게 고맙다&gt...,<b>★ ★ ★ ★ ★<BR>\n200만 독자가 사랑하는 '책 읽어주는 남자' 전승...,,10,[100자평]정말 예쁜 책. 어느 장을 펼쳐도 예쁜 모습을 하고 있...,chanel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1944,호찌냥찌 - 야생에서 쫓겨난 호랑이 호찌와 유쾌한 일곱 고양이들의 한집 살림 이야기,Grace J(정하나) (지은이),2019-08-23,"어느 날 우연히 숲속을 지나가는데, 아늑한 오두막집 앞에서 말하는 고양이와 사과를 ...",9791189969196,51376,국내도서>에세이>사진/그림 에세이,1623,False,10,"어느 날 우연히 숲속을 지나가는데, 아늑한 오두막집 앞에서 말하는 고양이와 사과를 ...",<b>〈라이온 킹〉만큼 경이롭고 감동적인 호랑이 삼촌 이야기<BR>\n“어른이 되어...,,10,[100자평]새로운 이야기 읽고 샀는데 역시 좋아요! 다음 시리즈도...,plum
1945,스위스의 고양이 사다리,"브리기테 슈스터 (지은이), 김목인 (옮긴이)",2021-01-15,애묘인 인구가 다수에 속하는 스위스에서 쉽게 마주칠 수 있는 ‘고양이 사다리’의 다...,9788986022285,51063,국내도서>예술/대중문화>사진>사진집,1620,False,8,애묘인 인구가 다수에 속하는 스위스에서 쉽게 마주칠 수 있는 ‘고양이 사다리’의 다...,애묘인 인구가 다수에 속하는 스위스에서 쉽게 마주칠 수 있는 ‘고양이 사다리’의 다...,,6,[마이리뷰]스위스의 고양이 사다리,dollc
1946,나는 당신이 살았으면 좋겠습니다 - 조울병 의사가 들려주는 조울병 이야기,안경희 (지은이),2018-02-08,조울병으로 '사회적 자살'에 이르렀던 저자가 자신의 경험을 바탕으로 쓴 에세이다. ...,9791187192787,51371,국내도서>에세이>한국에세이,1619,False,9,조울병 의사가 말하는 조울병에 관한 모든 것. 조울병으로 '사회적 자살'에 이르렀던...,<b>나는 왜 이렇게 감정 기복이 심할까? <BR>\n매일매일 널뛰는 감정으로 힘든...,,6,[100자평]저자의 진솔한 마음이 담긴 도서,stepout88
1947,우아한 가난의 시대 - 2020 문학나눔 선정도서,김지선 (지은이),2020-07-01,"""생각해 보면 지금까지는 엄살이었던 거고, 앞으로는 실제로 가난해질 확률이 너무나 ...",9791163641179,51371,국내도서>에세이>한국에세이,1617,False,9,<b>한국문화예술위원회 2020 하반기 문학나눔 도서보급사업에 선정된 도서입니다.<...,,,10,[마이리뷰]풍요로움과 가난함의 차이란,가필드


In [28]:
df.to_pickle('essay.plk', protocol=4)

In [233]:
df = pd.DataFrame(results)
df.columns = ['ISBN']
#df.to_excel('youtube_rank.xlsx', index=False)
df

,ISBN
0,9791161571188
1,9791197377143
2,9788936424725
3,9791190254168
4,9791191043617
5,9788954685436
6,9791165341909
7,9791191824001
8,9788946422056
9,9791165343729


In [167]:
results = []
w_lists = []
m_lists = []
buys = []
clicks = []

for page in range(1):
    url = "https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=Detail\
    &PublishMonth=0&SortOrder=2&page={}&Stockstatus=0&PublishDay=84&CustReviewRankStart=0&CustReviewCountStart=0&PriceFilterMax=-1\
    &CID=1&SearchOption=".format(page)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('ul > li > a.bo3')
    links = []
    for book_link in book_links:
        a= book_link['href']
        links.append(a)
    for link in links:
        driver.get(link)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            ISBN = soup.select('div.conts_info_list1 > ul > li')
            ISBN = ISBN[-1].text[-13:]
        except:
            pass
        try:
            for sex in range(6):
                woman_list = soup.select('tbody > tr > td.tb_woman')
                w_list = woman_list[sex].text.strip()
                w_lists.append(w_list)
                man_list = soup.select('tbody > tr > td.tb_man')
                m_list = man_list[sex].text.strip()
                m_lists.append(m_list)
        except:
            pass
        try:
            for rec in range(6):
                buy = soup.select('div #swiper_nowBuy dd > a')
                buy = buy[rec].text
                buys.append(buy)
                click = soup.select('div #swiper_nowClick dd > a')
                click = click[rec].text
                clicks.append(click)
        except:
            pass
        data = [ISBN, w_lists, man_list, buys, clicks]
        results.append(data)

In [212]:
results = []
w_lists = []
m_lists = []
buys = []
clicks = []

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
try:
    ISBN = soup.select('div.conts_info_list1 > ul > li')
    ISBN = ISBN[-1].text[-13:]
except:
    pass
try:
    for sex in range(6):
        woman_list = soup.select('tbody > tr > td.tb_woman')
        w_list = woman_list[sex].text.strip()
        w_lists.append(w_list)
        man_list = soup.select('tbody > tr > td.tb_man')
        m_list = man_list[sex].text.strip()
        m_lists.append(m_list)
except:
    pass
try:
    for rec in range(6):
        buy = soup.select('div #swiper_nowBuy dd > a')
        buy = buy[rec].text
        buys.append(buy)
        click = soup.select('div #swiper_nowClick dd > a')
        click = click[rec].text
        clicks.append(click)
except:
    pass
data = [ISBN, w_lists, man_list, buys, clicks]
results.append(data)

In [213]:
df = pd.DataFrame(results)
df.columns = ['ISBN', 'w_lists', 'm_lists', 'buys', 'click']
#df.to_excel('youtube_rank.xlsx', index=False)
df

,ISBN,w_lists,m_lists,buys,click
0,9791161571188,"[2.3%, 9.7%, 15.6%, 32.8%, 12.6%, 1.4%]","[[[<img src=""//image.aladin.co.kr/img/shop/201...","[어서 오세요, 휴남동 서점입니다, 마음에 없는 소리, 달러구트 꿈 백화점 (100...","[범, 하지 말지어다 1, 므레모사, 사내 맞선 1, 아침은 생각한다, 여덟 번째 ..."


In [72]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [39]:
# 페이지에 있는 도서 링크(50개)
book_links = soup.select('ul > li > a.bo3')
links = []
for book_link in book_links:
    a= book_link['href']
    links.append(a)
print(links)

['https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=290457417', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=286859586', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=290833437', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=289034480', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=288756034', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=290202593', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=245127051', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=276926308', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=289661123', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=275874708', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=289768950', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=275419192', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=290846875', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=290202822', 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=274077938', 'https://www.aladin.co.kr/shop/wproduct

In [74]:
links[0], links[1]

('https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=290457417',
 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=286859586')

In [33]:
len(channel_list)

50

In [ ]:
url = link
driver.get(html)

In [215]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [229]:
soup.select('div.ss_book_list b')

[<b>불편한 편의점 (40만부 기념 벚꽃 에디션)</b>,
 <b><span class="">12,600</span>원</b>,
 <b> 316,996</b>,
 <b>어서 오세요, 휴남동 서점입니다</b>,
 <b><span class="">13,500</span>원</b>,
 <b> 85,915</b>,
 <b>일하고 일하고 사랑을 하고</b>,
 <b><span class="">8,100</span>원</b>,
 <b> 35,250</b>,
 <b>진화 신화</b>,
 <b><span class="">10,800</span>원</b>,
 <b> 6,015</b>,
 <b>백광</b>,
 <b><span class="">13,050</span>원</b>,
 <b> 40,320</b>,
 <b>마음에 없는 소리</b>,
 <b><span class="">13,050</span>원</b>,
 <b> 16,990</b>,
 <b>달러구트 꿈 백화점</b>,
 <b><span class="">12,420</span>원</b>,
 <b> 332,842</b>,
 <b>지구 끝의 온실</b>,
 <b><span class="">13,500</span>원</b>,
 <b> 137,910</b>,
 <b>꽃잎 한 장처럼</b>,
 <b><span class="">14,400</span>원</b>,
 <b> 33,470</b>,
 <b>달러구트 꿈 백화점 2</b>,
 <b><span class="">12,420</span>원</b>,
 <b> 193,048</b>,
 <b>실마릴리온 + 끝나지 않은 이야기 세트 - 전2권</b>,
 <b><span class="">99,000</span>원</b>,
 <b> 40,050</b>,
 <b>밝은 밤</b>,
 <b><span class="">13,050</span>원</b>,
 <b> 150,382</b>,
 <b>저주토끼 (리커버)</b>,
 <b><span class="">13,320</span>원</b>,
 <b> 7

In [231]:
title = []
for i in range(150):
    if i % 3 == 0:
        text = soup.select('div.ss_book_list b')[i].text
        title.append(text)
title

['불편한 편의점 (40만부 기념 벚꽃 에디션)',
 '어서 오세요, 휴남동 서점입니다',
 '일하고 일하고 사랑을 하고',
 '진화 신화',
 '백광',
 '마음에 없는 소리',
 '달러구트 꿈 백화점',
 '지구 끝의 온실',
 '꽃잎 한 장처럼',
 '달러구트 꿈 백화점 2',
 '실마릴리온 + 끝나지 않은 이야기 세트 - 전2권',
 '밝은 밤',
 '저주토끼 (리커버)',
 '페스트의 밤',
 '오늘 밤, 세계에서 이 사랑이 사라진다 해도',
 '칵테일, 러브, 좀비',
 '소년이 온다',
 '대도시의 사랑법',
 '가운데땅의 지도들 세트',
 '시맨틱 에러 세트 - 전2권',
 '아몬드 (양장)',
 '2021 제12회 젊은작가상 수상작품집',
 '인간 실격',
 '어서 오세요 실력지상주의 교실에 2학년 편 5',
 '미드나잇 라이브러리',
 '우리가 빛의 속도로 갈 수 없다면',
 '키르케',
 '리커버 특별판 보러 가기>>',
 ' 62,260',
 ' 69,983',
 ' 17,540',
 ' 93,171',
 ' 9,460',
 ' 47,823',
 ' 83,204',
 ' 56,448',
 ' 60,482',
 ' 83,469',
 ' 43,570',
 ' 4,350',
 ' 12,980',
 '11,520원',
 '16,200원',
 '11,700원',
 '13,500원',
 '9,000원',
 '13,500원',
 '13,320원',
 '11,700원',
 '7,200원']

In [193]:
# ISBN
ISBN = soup.select('div.conts_info_list1 > ul > li')
ISBN[-1].text[-13:]

'9791161571188'

In [192]:
# 여자 분포
w_lists = []
for i in range(6):
    woman_list = soup.select('tbody > tr > td.tb_woman')
    w_list = woman_list[i].text.strip()
    w_lists.append(w_list)
w_lists

['2.3%', '9.7%', '15.6%', '32.8%', '12.6%', '1.4%']

In [191]:
man_list = soup.select('td.tb_man')
man_list

[<td class="tb_man"><div class="bar" id="bar_m_0" style="width: 1.74%;"><img src="//image.aladin.co.kr/img/shop/2018/analysis_bg1.png"/></div> <div class="per">1.1%</div></td>,
 <td class="tb_man"><div class="bar" id="bar_m_1" style="width: 10.75%;"><img src="//image.aladin.co.kr/img/shop/2018/analysis_bg1.png"/></div> <div class="per">6.8%</div></td>,
 <td class="tb_man"><div class="bar" id="bar_m_2" style="width: 5.95%;"><img src="//image.aladin.co.kr/img/shop/2018/analysis_bg1.png"/></div> <div class="per">3.7%</div></td>,
 <td class="tb_man"><div class="bar" id="bar_m_3" style="width: 13.21%;"><img src="//image.aladin.co.kr/img/shop/2018/analysis_bg1.png"/></div> <div class="per">8.3%</div></td>,
 <td class="tb_man"><div class="bar" id="bar_m_4" style="width: 7.41%;"><img src="//image.aladin.co.kr/img/shop/2018/analysis_bg1.png"/></div> <div class="per">4.7%</div></td>,
 <td class="tb_man"><div class="bar" id="bar_m_5" style="width: 1.74%;"><img src="//image.aladin.co.kr/img/shop/2

In [214]:
# 남자 분포
man_list = soup.select('tbody > tr > td.tb_man')
man_list

[<td class="tb_man"><div class="bar" id="bar_m_0" style="width: 1.74%;"><img src="//image.aladin.co.kr/img/shop/2018/analysis_bg1.png"/></div> <div class="per">1.1%</div></td>,
 <td class="tb_man"><div class="bar" id="bar_m_1" style="width: 10.75%;"><img src="//image.aladin.co.kr/img/shop/2018/analysis_bg1.png"/></div> <div class="per">6.8%</div></td>,
 <td class="tb_man"><div class="bar" id="bar_m_2" style="width: 5.95%;"><img src="//image.aladin.co.kr/img/shop/2018/analysis_bg1.png"/></div> <div class="per">3.7%</div></td>,
 <td class="tb_man"><div class="bar" id="bar_m_3" style="width: 13.21%;"><img src="//image.aladin.co.kr/img/shop/2018/analysis_bg1.png"/></div> <div class="per">8.3%</div></td>,
 <td class="tb_man"><div class="bar" id="bar_m_4" style="width: 7.41%;"><img src="//image.aladin.co.kr/img/shop/2018/analysis_bg1.png"/></div> <div class="per">4.7%</div></td>,
 <td class="tb_man"><div class="bar" id="bar_m_5" style="width: 1.74%;"><img src="//image.aladin.co.kr/img/shop/2

In [198]:
# 함께 구매한 책
buy = soup.select('div #swiper_nowBuy dd > a')
buy

[<a href="https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=286859586&amp;ptid=5&amp;utm_source=aladin&amp;utm_medium=wizard&amp;utm_campaign=relbuy&amp;utm_content=product">어서 오세요, 휴남동 서점입니다</a>,
 <a href="https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=290202593&amp;ptid=5&amp;utm_source=aladin&amp;utm_medium=wizard&amp;utm_campaign=relbuy&amp;utm_content=product">마음에 없는 소리</a>,
 <a href="https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=284135892&amp;ptid=5&amp;utm_source=aladin&amp;utm_medium=wizard&amp;utm_campaign=relbuy&amp;utm_content=product">달러구트 꿈 백화점 (100만부 기념 합본호 기프트 에디션)</a>,
 <a href="https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=275874708&amp;ptid=5&amp;utm_source=aladin&amp;utm_medium=wizard&amp;utm_campaign=relbuy&amp;utm_content=product">달러구트 꿈 백화점 2</a>,
 <a href="https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=276926308&amp;ptid=5&amp;utm_source=aladin&amp;utm_medium=wizard&amp;utm_campaign=relbuy&amp;utm_content=product">지구 끝의 온실</a>,
 <a href="https:

In [199]:
buy = buy[0].text
buy

'어서 오세요, 휴남동 서점입니다'

In [120]:
# 함께 클릭한 책
click = soup.select('div #swiper_nowClick dd > a')
click[0].text

'어서 오세요, 휴남동 서점입니다'